In [ ]:
import datetime
import operator
import matplotlib.pyplot as plt
from ipywidgets import interact
from IPython.display import Markdown as md
from datetime import date, timedelta, datetime
from numpy import sqrt
from pandas import read_csv, to_datetime, DataFrame, concat

# import pandas as pd
from prettytable import PrettyTable
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from google.colab import drive

drive.mount("/content/drive")
csv_file_path = "/content/drive/MyDrive/UPH/CLUSTERING/covid.csv"

try:
    covid = read_csv(csv_file_path)

except Exception as e:
    print(f"An error occurred: {e}")
covid = covid[covid["location"] == "Indonesia"]
covid = covid[["date", "total_cases", "location"]]
covid["date"] = to_datetime(covid["date"]).dt.strftime("%m/%d/%Y")
covid["total_cases"] = covid["total_cases"].astype(int)
covid = covid.reset_index(drop=True)
covid["indeks"] = covid.index
covid = covid[["indeks", "date", "total_cases", "location"]]
covid.columns = ["indeks", "tanggal", "kasus", "lokasi"]
last_date = to_datetime(covid["tanggal"].iloc[-1], format="%m/%d/%Y")
print(last_date)

degree = 2

x = covid["indeks"].values.reshape(covid["indeks"].size, 1)
y = covid["kasus"].values.reshape(covid["kasus"].size, 1)
polynomial_features = PolynomialFeatures(degree=degree)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = sqrt(mean_squared_error(y, y_poly_pred))
r2 = r2_score(y, y_poly_pred)


def forecast_days_after(n_days_after):
    # mengembalikan prediksi model regresi polynomial
    model_pred = model.predict(polynomial_features.fit_transform([[n_days_after]]))
    return "{:d}".format(int(round(model_pred[0][0])))


start_date = to_datetime(covid["tanggal"].iloc[0], format="%m/%d/%Y")
last_date = to_datetime(covid["tanggal"].iloc[-1], format="%m/%d/%Y")


def display_forecast_days_after(n_days_after):
    # mengembalikan prediksi model regresi polynomial
    end_date = start_date + timedelta(days=n_days_after)
    model_pred = model.predict(polynomial_features.fit_transform([[n_days_after]]))
    return end_date.strftime("%d %B %Y"), "{:d}".format(int(round(model_pred[0][0])))


def days_passed_timedelta():
    # mengembalikan hasil objek timedelta
    # berapa hari telah lewat sejak tanggal awal (start_date)
    last_date = to_datetime(covid["tanggal"].iloc[-1], format="%m/%d/%Y")
    return last_date.date() - start_date.date()


def date_days_after(n_days_after, date_format="%d %B %Y", **kwargs):
    tanggal = start_date + timedelta(days=n_days_after)
    return tanggal.date().strftime(date_format)


def display_tabel_prediksi(days_after):
    # tabel prediksi
    days_passed = days_passed_timedelta().days
    tabel_prediksi = PrettyTable(["ke-", "tanggal", "prediksi"])
    for x in range(days_passed + 1, days_after + 1):
        tabel_prediksi.add_row(
            [x, date_days_after(x, date_format="%d-%m-%Y"), forecast_days_after(x)]
        )
    print(tabel_prediksi)


def display_tabel_realisasi(lookback):
    # tabel prediksi
    days_passed = days_passed_timedelta().days
    tabel_realisasi = PrettyTable(["tanggal", "pred", "real", "p-r"])
    for x in range(days_passed, days_passed - lookback + 1, -1):
        pred_ = int(forecast_days_after(x))
        real_ = y[x][0]
        eval_ = pred_ - real_
        tabel_realisasi.add_row(
            [date_days_after(x, date_format="%d-%m-%Y"), pred_, real_, eval_]
        )
    print(tabel_realisasi)


def display_equation(model):
    for coef in model.coef_:
        pass
    streq = "$y = " + str(model.intercept_[0])
    for i, c in enumerate(coef):
        j = len(coef) - i - 1
        if abs(c) > c:
            sign = "-"
        else:
            sign = "+"
        if i > 1:
            streq += sign + str(abs(c)) + " \cdot x^{" + str(i) + "}"
        elif i == 1:
            streq += sign + str(abs(c)) + " \cdot x"

    streq = streq + "$"
    return md(streq)


def display_widget_forecast_days_after():
    print(
        "data terakhir adalah data hari ke-"
        + str(days_passed_timedelta().days)
        + ", untuk mendapatkan prediksi ke depan geser slider ke kanan (gunakan tombol panah kanan -> pada keyboard):"
    )

    @interact(x=(days_passed_timedelta().days + 1, 1000, 1))
    def h(x=days_passed_timedelta().days + 1):
        tabel_prediksi = PrettyTable(["hari ke-x", "tanggal", "prediksi akumulasi"])
        tabel_prediksi.add_row([x, date_days_after(x), forecast_days_after(x)])
        print(tabel_prediksi)

# tabel parameter
tabel_parameter = PrettyTable(["parameter", "nilai"])
tabel_parameter.add_row(["polynomial\ndegree", degree])
tabel_parameter.add_row(["RMSE", "{:.10}".format(rmse)])
tabel_parameter.add_row(["R^2", "{:.10}".format(r2)])

plt.scatter(x, y, s=10)
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x, y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.title("kumulatif kasus covid-19 di Indonesia\n")
plt.xlabel("\nhari ke-n sejak 1 Maret 2020")
plt.ylabel("jumlah akumulasi\nkonfirmasi positif")

plt.plot(x, y_poly_pred, color="m")
plt.show()
print(tabel_parameter)
display_equation(model)

# tabel prediksi hingga hari ke 60
display_tabel_prediksi(945)

display_widget_forecast_days_after()
display_forecast_days_after(946)

# tabel prediksi vs realisasi hingga 21 hari dari hari terakhir prediksi
# p - r = negatif pertanda situasi lebih buruk daripada perkiraan
# p - r = positif pertanda situasi lebih baik daripada perkiraan
display_tabel_realisasi(21)